# Problem 4.2

## part a

In [1]:
qui import delim ./data/chapter04/Gendelman_2004, clear
set seed 1234567
rename treatment D
rename posttest Y
rename pretest X

In [2]:
capture program drop Fstat
program define Fstat, rclass
	regress D X
    return scalar Fs=e(F)
end

// calculate 48620 (18 choose 9) permutations
tsrtest D r(Fs) using 4_2_Fstat.dta, overwrite: Fstat




Two-sample randomization test for theta=r(Fs) of Fstat by D

Combinations:   48620 = (18 choose 9)
Assuming null=0
Observed theta: 1.274

Minimum time needed for exact test (h:m:s):  0:03:27
Mode: exact

progress: |........................................|

 p=0.31345 [one-tailed test of Ho:  theta(D==0)<=theta(D==1)]
 p=0.76851 [one-tailed test of Ho:  theta(D==0)>=theta(D==1)]
 p=0.31345 [two-tailed test of Ho:  theta(D==0)==theta(D==1)]

Saving log file to 4_2_Fstat.dta...done.


In [3]:
//ate
di r(obsvStat)

1.2743363


In [4]:
// p.valueb
di r(uppertail)

.31345125


## part b

In [5]:
// calculate ate
qui reg Y D
global tau = _b[D]
di "ATE = " $tau




ATE = -.33333333


In [6]:
// RI under the null ate=ate
qui gen Y0_sim = Y
qui gen Y1_sim = Y
qui gen Y_sim = .
qui replace Y0_sim = Y - $tau if D==1
qui replace Y1_sim = Y + $tau if D==0
	
capture program drop ate_ci
program define ate_ci, rclass
	replace Y_sim = Y0_sim*(1-D) + Y1_sim*(D) 
	regress Y_sim D 
    return scalar Ys=_b[D]	
end

tsrtest D r(Ys) using ate_ci.dta, overwrite: ate_ci









Two-sample randomization test for theta=r(Ys) of ate_ci by D

Combinations:   48620 = (18 choose 9)
Assuming null=0
Observed theta: -.3333

Minimum time needed for exact test (h:m:s):  0:03:26
Mode: exact

progress: |........................................|

 p=0.50428 [one-tailed test of Ho:  theta(D==0)<=theta(D==1)]
 p=0.50000 [one-tailed test of Ho:  theta(D==0)>=theta(D==1)]
 p=0.76450 [two-tailed test of Ho:  theta(D==0)==theta(D==1)]

Saving log file to ate_ci.dta...done.


In [7]:
preserve
use "ate_ci.dta", clear
qui drop if _n==1

sort theta
// 95% confidence interval (CI)
di "(" round(theta[floor(_N*0.025)], 0.001) ", "round(theta[floor(_N*0.975)], 0.001) ")"

restore






(-2.259, 1.593)



## part c

In [8]:
rename improvement Y_improve 

// calculate ate.improve
qui reg Y_improve D
global tau_im = _b[D]
di "ATE.improve = " $tau_im





ATE.improve = 1


In [9]:
// RI under the null ate=ate.improve
qui replace Y0_sim = Y_improve
qui replace Y1_sim = Y_improve
qui replace Y_sim = .

qui replace Y0_sim = Y_improve - $tau_im if D==1
qui replace Y1_sim = Y_improve + $tau_im if D==0

In [10]:
capture program drop ate_im_ci
program define ate_im_ci, rclass
	replace Y_sim = Y0_sim*(1-D) + Y1_sim*(D) 
	regress Y_sim D 
    return scalar Ys_im=_b[D]	
end

tsrtest D r(Ys_im) using ate_im_ci.dta, overwrite: ate_im_ci




Two-sample randomization test for theta=r(Ys_im) of ate_im_ci by D

Combinations:   48620 = (18 choose 9)
Assuming null=0
Observed theta: 1

Minimum time needed for exact test (h:m:s):  0:03:29
Mode: exact

progress: |........................................|

 p=0.58739 [one-tailed test of Ho:  theta(D==0)<=theta(D==1)]
 p=0.59726 [one-tailed test of Ho:  theta(D==0)>=theta(D==1)]
 p=0.58739 [two-tailed test of Ho:  theta(D==0)==theta(D==1)]

Saving log file to ate_im_ci.dta...done.


In [11]:
use "ate_im_ci.dta", clear
drop if _n==1

sort theta

// 95% confidence interval (CI)

// 95% CI is different from R result due to rounding
// the permutation test is exactly the same

di "95%CI = " "(" round(theta[floor(_N*0.025)], 0.001) " , " round(theta[floor(_N*0.975)], 0.001) ")"



(1 observation deleted)


95%CI = (.111 , 1.889)
